# Enviroment setup

## Spark Session enviroment setup

In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Football_data_analysis").getOrCreate()

In [2]:
spark

## Loading and checking data

In [ ]:
matches_df = spark.read.csv("s3://databricks-aws-spartans/football_data_analysis_spark_project/source_file/matches.csv", header=True, inferSchema=True)
players_df = spark.read.csv("s3://databricks-aws-spartans/football_data_analysis_spark_project/source_file/players.csv", header=True, inferSchema=True)


In [4]:
# columns = date_dim_df.columns
# print(" | ".join(columns))

In [5]:
columns = players_df.columns
print(" | ".join(columns))

Year | Month | Match | Player Name | Club Name | country Name | Position | Jersey_No | D. O. B. | Nationality | appearances | goals scored | goals assist | total shots | shots on target | fouls made | fouls suffered | yellow card | red card | goals saved | goals conceded(stopped) | total penalty | successful penalty | salary


In [6]:
columns = matches_df.columns
print(" | ".join(columns))

Year | Match_Name | League_Name | Club_Name | Coach | Country Name | Manager | Owner | appearances | wins | losts | drawn | clean sheets | Net Worth


## Little cleaning of data columns names and coulumns data

In [7]:
players_df = players_df.withColumnRenamed("D. O. B.", "DOB")

In [8]:
players_df = players_df.withColumnRenamed("goals conceded(stopped)", "goals conceded")

In [9]:
matches_df = matches_df.withColumnRenamed("Match_Name", "match")

In [10]:
# Get current column names
current_columns = players_df.columns

# Create a new list of column names with underscores replaced by spaces
new_columns = [col_name.replace("_", " ").lower() for col_name in current_columns]

# Rename columns in the DataFrame
for old_name, new_name in zip(current_columns, new_columns):
    players_df = players_df.withColumnRenamed(old_name, new_name)

# Show the updated DataFrame
columns = players_df.columns
print(" | ".join(columns))

year | month | match | player name | club name | country name | position | jersey no | dob | nationality | appearances | goals scored | goals assist | total shots | shots on target | fouls made | fouls suffered | yellow card | red card | goals saved | goals conceded | total penalty | successful penalty | salary


In [11]:
# Get current column names
current_columns = matches_df.columns

# Create a new list of column names with underscores replaced by spaces
new_columns = [col_name.replace("_", " ").lower() for col_name in current_columns]

# Rename columns in the DataFrame
for old_name, new_name in zip(current_columns, new_columns):
    matches_df = matches_df.withColumnRenamed(old_name, new_name)

# Show the updated DataFrame
columns = matches_df.columns
print(" | ".join(columns))

year | match | league name | club name | coach | country name | manager | owner | appearances | wins | losts | drawn | clean sheets | net worth


In [12]:
from pyspark.sql.functions import when, col

players_df = players_df.withColumn("club name", when(col("club name") == "NA", None).otherwise(col("club name")))
players_df = players_df.withColumn("country name", when(col("country name") == "NA", None).otherwise(col("country name")))

matches_df = matches_df.withColumn("league name", when(col("league name") == "NA", None).otherwise(col("league name")))
matches_df = matches_df.withColumn("club name", when(col("club name") == "NA", None).otherwise(col("club name")))
matches_df = matches_df.withColumn("coach", when(col("coach") == "NA", None).otherwise(col("coach")))
matches_df = matches_df.withColumn("country name", when(col("country name") == "NA", None).otherwise(col("country name")))
matches_df = matches_df.withColumn("manager", when(col("manager") == "NA", None).otherwise(col("manager")))
matches_df = matches_df.withColumn("owner", when(col("owner") == "NA", None).otherwise(col("manager")))

In [13]:
from pyspark.sql.functions import regexp_replace

players_df = players_df \
    .withColumn("player name", regexp_replace("player name", "Thiago Alc�ntara", "Thiago Alcantara"))

## Printing schema to undestand data

In [14]:
print("Schema for matches statistics:")
matches_df.printSchema()

Schema for matches statistics:
root
 |-- year: integer (nullable = true)
 |-- match: string (nullable = true)
 |-- league name: string (nullable = true)
 |-- club name: string (nullable = true)
 |-- coach: string (nullable = true)
 |-- country name: string (nullable = true)
 |-- manager: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- appearances: integer (nullable = true)
 |-- wins: integer (nullable = true)
 |-- losts: integer (nullable = true)
 |-- drawn: integer (nullable = true)
 |-- clean sheets: integer (nullable = true)
 |-- net worth: string (nullable = true)



In [15]:
print("Schema for player statistics:")
players_df.printSchema()

Schema for player statistics:
root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- match: string (nullable = true)
 |-- player name: string (nullable = true)
 |-- club name: string (nullable = true)
 |-- country name: string (nullable = true)
 |-- position: string (nullable = true)
 |-- jersey no: integer (nullable = true)
 |-- dob: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- appearances: integer (nullable = true)
 |-- goals scored: integer (nullable = true)
 |-- goals assist: integer (nullable = true)
 |-- total shots: integer (nullable = true)
 |-- shots on target: integer (nullable = true)
 |-- fouls made: integer (nullable = true)
 |-- fouls suffered: integer (nullable = true)
 |-- yellow card: integer (nullable = true)
 |-- red card: integer (nullable = true)
 |-- goals saved: integer (nullable = true)
 |-- goals conceded: integer (nullable = true)
 |-- total penalty: integer (nullable = true)
 |-- successful penalty: int

Changing datatype of dob from string to date

In [16]:
players_df = players_df.withColumn("dob", col("dob").cast("date"))

In [17]:
matches_df.count()

1440

In [18]:
players_df.count()

10800

In [19]:
players_df.describe()

DataFrame[summary: string, year: string, month: string, match: string, player name: string, club name: string, country name: string, position: string, jersey no: string, nationality: string, appearances: string, goals scored: string, goals assist: string, total shots: string, shots on target: string, fouls made: string, fouls suffered: string, yellow card: string, red card: string, goals saved: string, goals conceded: string, total penalty: string, successful penalty: string, salary: string]

In [20]:
matches_df.describe()

DataFrame[summary: string, year: string, match: string, league name: string, club name: string, coach: string, country name: string, manager: string, owner: string, appearances: string, wins: string, losts: string, drawn: string, clean sheets: string, net worth: string]

## Creating Dimension Tables

### 1. Dim_players
Dimension table of players

In [21]:
from pyspark.sql.functions import monotonically_increasing_id

# dim_players = players_df.filter(players_df["player name"] != None)

dim_players = players_df.dropDuplicates(["player name"]).orderBy("player name").select(
    monotonically_increasing_id().alias("player id"),
    "player name",
    "dob",
    "nationality",
    "position",
    "jersey no",
)

In [22]:
dim_players.show(6, True)
# dim_players.write.csv(r"projectCSV/dim_players", header=True)

+---------+--------------------+----+-----------+--------+---------+
|player id|         player name| dob|nationality|position|jersey no|
+---------+--------------------+----+-----------+--------+---------+
|        0|              Adrian|NULL|Netherlands|      LW|       88|
|        1|              Aguero|NULL|      Italy|       F|       51|
|        2|           Alberto M|NULL|     Brasil|      LB|       74|
|        3|Alessandro  Florenzi|NULL|   Portugal|      RW|       26|
|        4|       Alessio Cerci|NULL|      Italy|       F|       84|
|        5|    Alex Chamberlain|NULL|      Italy|      RW|       59|
+---------+--------------------+----+-----------+--------+---------+
only showing top 6 rows



### 2. Dim_clubs
Dimension table of clubs

In [23]:
# Drop the which has NA in it.
# dim_clubs = players_df.filter(players_df["club name"] != None)

dim_clubs = players_df.dropDuplicates(["club name"]).orderBy("club name").select(
    monotonically_increasing_id().alias("club id"),
    "club name",
)

In [24]:
dim_clubs.show(6, True)
# dim_clubs.write.csv(r"projectCSV/dim_clubs", header=True)

+-------+------------------+
|club id|         club name|
+-------+------------------+
|      0|              NULL|
|      1| Atheletico Madrid|
|      2|           Arsenal|
|      3|       Aston Villa|
|      4| Atheletico Bilbao|
|      5|         Barcelona|
+-------+------------------+
only showing top 6 rows



### 3. Dim_matches
Dimension table of matches

In [25]:
# dim_matches = players_df.filter(players_df["match"] != None)

dim_matches = players_df.dropDuplicates(["match"]).orderBy("match").select(
    monotonically_increasing_id().alias("match id"),
    "match",
)

In [26]:
dim_matches.show(6, True)
# dim_matches.write.csv(r"projectCSV/dim_matches", header=True)

+--------+-------------+
|match id|        match|
+--------+-------------+
|       0|         Club|
|       1|International|
+--------+-------------+



### 4. Dim_countries
Dimension table of countries

In [27]:
# dim_countries = players_df.filter(players_df["country name"] != None)

dim_countries = players_df.dropDuplicates(["country name"]).orderBy("country name").select(
    monotonically_increasing_id().alias("country id"),
    "country name"
)

In [28]:
dim_countries.show(6, True)
# dim_countries.write.csv(r"projectCSV/dim_countries", header=True)

+----------+------------+
|country id|country name|
+----------+------------+
|         0|        NULL|
|         1|   Argentina|
|         2|     Belgium|
|         3|      Brasil|
|         4|     England|
|         5|     Germany|
+----------+------------+
only showing top 6 rows



### 5. Dim_leagues
Dimension table of leagues

In [29]:
# dim_league = matches_df.filter(matches_df["league name"] != None)

dim_leagues = matches_df.dropDuplicates(["league name"]).orderBy("league name").select(
    monotonically_increasing_id().alias("league id"),
    "league name",
)

In [30]:
dim_leagues.show(6, True)
# dim_league.write.csv(r"projectCSV/dim_league", header=True)

+---------+-----------+
|league id|league name|
+---------+-----------+
|        0|       NULL|
|        1| Bundesliga|
|        2|        EPL|
|        3|    LA Liga|
+---------+-----------+



### 6. Dim_coaches
Dimension table of coaches

In [31]:
# dim_league = matches_df.filter(matches_df["coach"] != None)

dim_coaches = matches_df.dropDuplicates(["coach"]).orderBy("coach").select(
    monotonically_increasing_id().alias("coach id"),
    "coach",
)

In [32]:
dim_coaches.show(6, True)
# dim_coaches.write.csv(r"projectCSV/dim_coaches", header=True)

+--------+--------------+
|coach id|         coach|
+--------+--------------+
|       0|          NULL|
|       1|Barry Reynolds|
|       2| Cody Delacruz|
|       3|  Damon Pruitt|
|       4|  Elmo Patrick|
|       5|Kuame Gallegos|
+--------+--------------+
only showing top 6 rows



### 7. Dim_managers
Dimension table of managers

In [33]:
# dim_league = matches_df.filter(matches_df["manager"] != None)

dim_managers = matches_df.dropDuplicates(["manager"]).orderBy("manager").select(
    monotonically_increasing_id().alias("manager id"),
    "manager"
)

In [34]:
dim_managers.show(6, True)
# dim_managers.write.csv(r"projectCSV/dim_managers", header=True)

+----------+---------------+
|manager id|        manager|
+----------+---------------+
|         0|           NULL|
|         1|    Alvin Lucas|
|         2|    Axel Farley|
|         3|     Bruce Bray|
|         4|Caesar Townsend|
|         5|  Chava Rosales|
+----------+---------------+
only showing top 6 rows



### 8. Dim_owners
Dimension table of owners

In [35]:
# dim_league = matches_df.filter(matches_df["owner"] != None)

dim_owners = matches_df.dropDuplicates(["owner"]).orderBy("owner").select(
    monotonically_increasing_id().alias("owner id"),
    "owner"
)

In [36]:
dim_owners.show(6, True)
# dim_owners.write.csv(r"projectCSV/dim_owners", header=True)

+--------+---------------+
|owner id|          owner|
+--------+---------------+
|       0|           NULL|
|       1|    Alvin Lucas|
|       2|    Axel Farley|
|       3|     Bruce Bray|
|       4|Caesar Townsend|
|       5|  Chava Rosales|
+--------+---------------+
only showing top 6 rows



### 9. Dim_months
Dimension table of months

In [37]:
# dim_league = players_df.filter(players_df["month"] != None)

dim_months = players_df.dropDuplicates(["month"]).orderBy("month").select(
    monotonically_increasing_id().alias("month id"),
    "month"
)

In [38]:
dim_months.show(6, True)
# dim_months.write.csv(r"projectCSV/dim_months", header=True)

+--------+--------+
|month id|   month|
+--------+--------+
|       0|   April|
|       1|  August|
|       2|December|
|       3|February|
|       4| January|
|       5|    July|
+--------+--------+
only showing top 6 rows



### Players_peformance_fact
Fact table of players

In [39]:
players_performance_fact = players_df.alias('p') \
    .join(dim_months.alias('m'), col('p.month') == col('m.month'), 'left') \
    .join(dim_matches.alias('match'), col('p.match') == col('match.match'), 'left') \
    .join(dim_players.alias('pl'), col('p.player name') == col('pl.player name'), 'left') \
    .join(dim_clubs.alias('c'), col('p.club name') == col('c.club name'), 'left') \
    .join(dim_countries.alias('co'), col('p.country name') == col('co.country name'), 'left') \
    .select(
        (monotonically_increasing_id()+1001).alias("fact id"),
        col('p.year'),
        col('month id'),
        col('match id'),
        col('player id'),
        col('club id'),
        col('country id'),
        col('p.appearances'),
        col('p.goals scored'),
        col('p.goals assist'),
        col('p.total shots'),
        col('p.shots on target'),
        col('p.fouls made'),
        col('p.fouls suffered'),
        col('p.yellow card'),
        col('p.red card'),
        col('p.goals saved'),
        col('p.goals conceded'),
        col('p.total penalty'),
        col('p.successful penalty'),
        col('p.salary')
    )

In [40]:
players_performance_fact.show(5, True)
# players_performance_fact.write.csv(r"projectCSV/players_performance_fact", header=True)

+-------+----+--------+--------+---------+-------+----------+-----------+------------+------------+-----------+---------------+----------+--------------+-----------+--------+-----------+--------------+-------------+------------------+---------+
|fact id|year|month id|match id|player id|club id|country id|appearances|goals scored|goals assist|total shots|shots on target|fouls made|fouls suffered|yellow card|red card|goals saved|goals conceded|total penalty|successful penalty|   salary|
+-------+----+--------+--------+---------+-------+----------+-----------+------------+------------+-----------+---------------+----------+--------------+-----------+--------+-----------+--------------+-------------+------------------+---------+
|   1001|2010|       4|       1|        0|   NULL|         8|          8|           8|           3|         67|             49|         8|             0|          0|       0|         29|            23|            2|                 1|$2,424.88|
|   1002|2010|      

### Match_performance_fact
Fact table of matches

In [41]:
match_performance_fact = matches_df.alias('mt') \
    .join(dim_matches.alias('dm'), col('mt.match') == col('dm.match'), 'left') \
    .join(dim_leagues.alias('lea'), col('mt.league name') == col('lea.league name'), 'left') \
    .join(dim_clubs.alias('cl'), col('mt.club name') == col('cl.club name'), 'left') \
    .join(dim_coaches.alias('coach'), col('mt.coach') == col('coach.coach'), 'left') \
    .join(dim_countries.alias('coun'), col('mt.country name') == col('coun.country name'), 'left') \
    .join(dim_managers.alias('mana'), col('mt.manager') == col('mana.manager'), 'left') \
    .join(dim_owners.alias('o'), col('mt.owner') == col('o.owner'), 'left') \
    .select(
        (monotonically_increasing_id()+1001).alias("fact id"),
        col('mt.year'),
        col('match id'),
        col('league id'),
        col('club id'),
        col('coach id'),
        col('country id'),
        col('manager id'),
        col('owner id'),
        col('mt.appearances'),
        col('mt.wins'),
        col('mt.losts'),
        col('mt.drawn'),
        col('mt.clean sheets'),
        col('mt.net worth')
    )

In [42]:
match_performance_fact.show(5, True)
# match_performance_fact.write.csv(r"projectCSV/match_performance_fact", header=True)

+-------+----+--------+---------+-------+--------+----------+----------+--------+-----------+----+-----+-----+------------+---------+
|fact id|year|match id|league id|club id|coach id|country id|manager id|owner id|appearances|wins|losts|drawn|clean sheets|net worth|
+-------+----+--------+---------+-------+--------+----------+----------+--------+-----------+----+-----+-----+------------+---------+
|   1001|2010|       0|        3|      1|    NULL|      NULL|         1|       1|          8|   7|    0|    1|           5|$745,634 |
|   1002|2010|       0|        3|      1|    NULL|      NULL|         1|       1|          8|   2|    1|    5|           1|$745,634 |
|   1003|2010|       0|        3|      1|    NULL|      NULL|         1|       1|          8|   6|    1|    1|           3|$745,634 |
|   1004|2010|       0|        3|      1|    NULL|      NULL|         1|       1|          8|   6|    1|    1|           4|$745,634 |
|   1005|2010|       0|        3|      1|    NULL|      NULL| 

In [43]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Club report of Total Wins , Total Appearances ,Total Losses ,Total Draws
### League and Competition Insights
Clubs can compare their performance against league averages or top performers, providing insights into areas for improvement.
Trend Analysis: Longitudinal analysis of match results and player performances over time can reveal trends that influence future strategies.
### Ownership and Sponsorship
The top companies can fund or sponsor on the clubs based on their performance

In [44]:
# Join the fact table with the dimension table
joined_df = match_performance_fact \
    .join(dim_clubs, match_performance_fact["club id"] == dim_clubs["club id"], "left")

joined_df = joined_df.filter(joined_df["club name"].isNotNull())

# Group by Club Name and aggregate performance metrics
team_performance = joined_df.groupBy("club name") \
    .agg({
        "wins": "sum",
        "losts": "sum",
        "drawn": "sum",
        "appearances": "sum"
    }) \
    .withColumnRenamed("sum(wins)", "Total Wins") \
    .withColumnRenamed("sum(losts)", "Total Losses") \
    .withColumnRenamed("sum(drawn)", "Total Draws") \
    .withColumnRenamed("sum(appearances)", "Total Appearances")

# Show the results
# team_performance.show(5, True)
team_performance.limit(5).show()
# team_performance.write.csv("s3://databricks-aws-spartans/football_data_analysis_spark_project/reports/club_reports.csv")

+------------------+----------+-----------------+------------+-----------+
|         club name|Total Wins|Total Appearances|Total Losses|Total Draws|
+------------------+----------+-----------------+------------+-----------+
| Atheletico Madrid|       148|              288|          64|         76|
|           Arsenal|       124|              288|          77|         87|
|       Aston Villa|       139|              288|          79|         70|
| Atheletico Bilbao|       114|              288|         103|         73|
|         Barcelona|       123|              288|         101|         64|
+------------------+----------+-----------------+------------+-----------+



## Player performance analysis
### Identify Top Players by Goals and Assists(Helpful for football clubs)
By examining player statistics (goals scored, assists, fouls), clubs can identify top-performing players and make informed decisions regarding contracts, buying or selling, player development.
### Populating Sports websites 
Sports websites can populate these datas in their platform to enhance the quality of football datas to the end users.

In [45]:
# Join the fact table with the dimension table
joined_df = players_performance_fact \
    .join(dim_players, players_performance_fact["player id"] == dim_players["player id"], "inner")

joined_df = joined_df.filter(joined_df["player name"].isNotNull())

# Group by Club Name and aggregate performance metrics
top_players = joined_df.groupBy("player name") \
    .agg({
        "goals scored": "sum",
        "goals assist": "sum",
        "appearances": "sum",
        "total shots": "sum",
        "shots on target": "sum",
        "goals saved": "sum",
        "total penalty": "sum",
        "successful penalty": "sum"
    }) \
    .withColumnRenamed("sum(goals scored)", "Total Goals") \
    .withColumnRenamed("sum(goals assist)", "Total Assists") \
    .withColumnRenamed("sum(appearances)", "Total Appearances") \
    .withColumnRenamed("sum(total shots)", "Total Shots") \
    .withColumnRenamed("sum(shots on target)", "Shots On Target") \
    .withColumnRenamed("sum(goals saved)", "Goals Saved") \
    .withColumnRenamed("sum(total penalty)", "Total Penalty") \
    .withColumnRenamed("sum(successful penalty)", "Successful Penalty") \
    .orderBy("Total Goals", ascending=False)

In [46]:
# top_players.show(5, True)
top_players.limit(10).show()
# top_players.write.csv("s3://databricks-aws-spartans/football_data_analysis_spark_project/reports/player_reports.csv")

+--------------+---------------+-----------------+-----------+-------------+-----------+------------------+-----------+-------------+
|   player name|Shots On Target|Total Appearances|Total Goals|Total Penalty|Total Shots|Successful Penalty|Goals Saved|Total Assists|
+--------------+---------------+-----------------+-----------+-------------+-----------+------------------+-----------+-------------+
|       Gabriel|           2880|              576|        483|          325|       4566|               210|       1404|          338|
|   Eden Hazard|           2859|              576|        473|          337|       4682|               205|       1439|          304|
|Tallulah Leach|           2974|              576|        471|          341|       4870|               211|       1440|          318|
|     Hernandez|           2861|              576|        468|          318|       4616|               204|       1501|          357|
| Iker Casillas|           2916|              576|        465|

In [47]:
top_players.filter(top_players["player name"] == "Eden Hazard").show()

+-----------+---------------+-----------------+-----------+-------------+-----------+------------------+-----------+-------------+
|player name|Shots On Target|Total Appearances|Total Goals|Total Penalty|Total Shots|Successful Penalty|Goals Saved|Total Assists|
+-----------+---------------+-----------------+-----------+-------------+-----------+------------------+-----------+-------------+
|Eden Hazard|           2859|              576|        473|          337|       4682|               205|       1439|          304|
+-----------+---------------+-----------------+-----------+-------------+-----------+------------------+-----------+-------------+



## Player Injury & Suspension Analysis
### Helpful for club
By tracking fouls committed, yellow/red cards (for goalkeepers), clubs can better understand a player's tendency for  suspensions. This information is vital for managing player health, avoiding suspensions, and understanding risks. The clubs can also avoid to extend contract with those player who have highest tendency to get a card.
### Helpful for sports website
They can populate these datas also in their website

In [48]:
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F

In [49]:
# Join the fact table with the dimension table
joined_df = players_performance_fact \
    .join(dim_players, players_performance_fact["player id"] == dim_players["player id"], "inner")

joined_df = joined_df.filter(joined_df["player name"].isNotNull())

# Ensure that the columns are cast to integer type before performing the sum operation
disciplinary_df = joined_df.groupBy("player name") \
.agg(
    F.sum(col("yellow card").cast(IntegerType())).alias("total yellow cards"),
    F.sum(col("red card").cast(IntegerType())).alias("total red cards"),
    F.sum(col("fouls made").cast(IntegerType())).alias("total fouls made")
)

# Order by total_fouls_made in descending order
ordered_disciplinary_df = disciplinary_df.orderBy(F.col("total fouls made").desc())

In [50]:
# Show the result
# ordered_disciplinary_df.show(6, True)
ordered_disciplinary_df.limit(10).show()
# ordered_disciplinary_df.write.csv("s3://databricks-aws-spartans/football_data_analysis_spark_project/reports/order_disciplinary_reports.csv")

+-----------------+------------------+---------------+----------------+
|      player name|total yellow cards|total red cards|total fouls made|
+-----------------+------------------+---------------+----------------+
|   Noah Whitfield|               142|             73|             479|
|         Fletcher|               157|             83|             474|
|             Hulk|               122|             85|             471|
|   Phelan Sampson|               138|             75|             466|
|      David Villa|               158|             79|             462|
|      Juan Manuel|               141|             65|             462|
|     Patrice Evra|               143|             68|             459|
|            Giggs|               146|             70|             457|
|     Philipp Lahm|               157|             60|             456|
|Javier Mascherano|               137|             68|             455|
+-----------------+------------------+---------------+----------

## Country-Specific Performance Insights
### Scouting project insights
By getting the country specific statistics the club can build a scouting project on those countries which are performing in the club levels as well as the international level


In [51]:
# Join the fact table with the dimension table
joined_df = players_performance_fact \
    .join(dim_countries, players_performance_fact["country id"] == dim_countries["country id"], "inner")

joined_df = joined_df.filter(joined_df["country name"].isNotNull())

# Group by country and calculate performance metrics
country_performance_df = joined_df.orderBy("country name").groupBy("country name").agg(
    F.sum("goals scored").alias("total goals"),
    F.sum("goals assist").alias("total assists")
)

In [52]:
# country_performance_df.show(5, True)
country_performance_df.show()
# country_performance_df.write.csv("s3://databricks-aws-spartans/football_data_analysis_spark_project/reports/country_performance_reports.csv")

+------------+-----------+-------------+
|country name|total goals|total assists|
+------------+-----------+-------------+
|   Argentina|       2852|         2494|
|     Belgium|       2891|         2390|
|      Brasil|       2868|         2340|
|     England|       2936|         2410|
|     Germany|       2976|         2403|
|       Italy|       2938|         2481|
|      Mexico|       2936|         2428|
| Netherlands|       2884|         2462|
|    Portugal|       2879|         2459|
|       Spain|       2904|         2408|
+------------+-----------+-------------+



##   find top goal scorers of a specific year
### Sports website data 
Sports websites can use the datas to populate their information as much as possible as the old football datas are not that much available in the market.

In [53]:
top_scorers_2010 = players_performance_fact.filter(players_performance_fact["year"] == 2010) \
    .groupBy("player id") \
    .agg(F.sum("goals scored").alias("total goals")) \
    .limit(10) \
    .join(dim_players, "player id") \
    .select("player id","player name", "total goals") \
    .orderBy("total goals", ascending=False)

In [54]:
# top_scorers_2010.show(6, True)
top_scorers_2010.show()
# top_scorers_2010.write.csv("s3://databricks-aws-spartans/football_data_analysis_spark_project/reports/top_scorer_2010.csv")

+---------+-----------------+-----------+
|player id|      player name|total goals|
+---------+-----------------+-----------+
|       54|        Hernandez|        155|
|        0|           Adrian|        138|
|       29|      David Villa|        136|
|       22|Cristiano Ronaldo|        132|
|       65|         Joe Hart|        132|
|       19|         Carvajal|        130|
|      113|   Raphael Varane|        126|
|       26|    Danny Welbeck|        125|
|      130|     Thiago Silva|        121|
|      112|          Ramires|        109|
+---------+-----------------+-----------+



## Find who has maximum appearances in International matches
### Brand Ambassadorship & Endorsements
#### Personal Brand: 
An experienced football player often has a strong personal brand, built over years of playing at a high level. This brand can be leveraged for endorsements, sponsorships, and partnerships with companies, making them valuable assets for brands looking to reach specific consumer segments.

#### Influence and Reach: 
Players with a large following on social media platforms can be influencers for brands, helping them engage with fans and customers on a personal level.

#### Product Partnerships: 
Football players often collaborate with companies for product endorsements in sectors like apparel, footwear, health supplements, or even technology.

In [55]:
Player_with_max_appearances = players_performance_fact.filter(players_performance_fact["country id"].isNotNull()) \
    .groupBy("player id", "country id") \
    .sum("appearances") \
    .orderBy("sum(appearances)", ascending=False) \
    .limit(1) \
    .join(dim_players, "player id") \
    .join(dim_countries, "country id") \
    .select("player id","player name", "sum(appearances)", "country name")

In [56]:
# Player_with_max_appearances.show()
Player_with_max_appearances.show()
# Player_with_max_appearances.write.csv("s3://databricks-aws-spartans/football_data_analysis_spark_project/reports/player_max_international_app.csv")

+---------+-----------+----------------+------------+
|player id|player name|sum(appearances)|country name|
+---------+-----------+----------------+------------+
|       95| Mesut Ozil|             288|     Belgium|
+---------+-----------+----------------+------------+

